In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
# importing LGBM function
from lightgbm import LGBMClassifier

In [3]:
# loading in data
df = pd.read_csv("train.csv")

df["target_change"] = np.ones_like(df["target"])

df.loc[df["target"]<0,'target_change'] = -1

df = df.drop(["target","bid_price","ask_price","row_id"], axis=1)

# trying a thing
st0 = df[df["stock_id"] == 0]

st0 = st0.drop(["stock_id","time_id","reference_price","far_price","near_price","date_id"], axis = 1)

st0.head()

,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,matched_size,bid_size,ask_size,wap,target_change
0,0,3180602.69,1,13380276.64,60651.50,8493.03,1.000000,-1.0
191,10,1299772.70,1,15261106.63,13996.50,23519.16,0.999892,1.0
382,20,1299772.70,1,15261106.63,4665.50,12131.60,0.999842,1.0
573,30,1299772.70,1,15261106.63,55998.00,46203.30,1.000085,1.0
764,40,1218204.43,1,15342674.90,14655.95,26610.45,1.000317,1.0


In [4]:
# creating test-train split
X = st0.drop(["target_change"], axis=1)

# created response variable array
y = st0["target_change"]

X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.2)

In [5]:
# creating model with fit
lgbmod = LGBMClassifier()
lgbmod.fit(X_tr,y_tr)

[LightGBM] [Info] Number of positive: 10346, number of negative: 10818
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001021 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1333
[LightGBM] [Info] Number of data points in the train set: 21164, number of used features: 7
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.488849 -> initscore=-0.044611
[LightGBM] [Info] Start training from score -0.044611


LGBMClassifier()

In [6]:
# evaluating accuracy of model
accuracy = lgbmod.score(X_tst, y_tst)
print(accuracy)

0.6038556038556039


In [7]:
y_hat = lgbmod.predict(X_tst)

In [10]:
lgbm_score = accuracy_score(y_tst, y_hat)
lgbm_conf_mat = confusion_matrix(y_tst, y_hat)
print(lgbm_score)
print(lgbm_conf_mat)


0.6038556038556039
[[1757  932]
 [1164 1438]]


Trying the feature selection I found from model selection using baseline model

In [16]:
# creating test-train split
X = st0[['seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag']]

# created response variable array
y = st0["target_change"]

X_tr, X_tst, y_tr, y_tst = train_test_split(X, y, test_size=0.2)

# creating model with fit
lgbmod = LGBMClassifier()
lgbmod.fit(X_tr,y_tr)

lgbm_score = accuracy_score(y_tst, y_hat)
lgbm_conf_mat = confusion_matrix(y_tst, y_hat)
print(lgbm_score)
print(lgbm_conf_mat)

[LightGBM] [Info] Number of positive: 10328, number of negative: 10836
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001742 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 313
[LightGBM] [Info] Number of data points in the train set: 21164, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.487998 -> initscore=-0.048015
[LightGBM] [Info] Start training from score -0.048015
0.5116235116235116
[[1504 1167]
 [1417 1203]]
